# Building a Baseline Model for MBTI Predictions

## Data Preparation 

### 1) What is our dataset?

Our dataset that we will be utilizing for this baseline model is the 'Types 500' MBTI dataset,'(MBTI) Myers-Briggs Personality Type Dataset', and the 'MBTI Personality Type Twitter Dataset' from Kaggle.

###  GAN